## OpenAI API - test connection

In [3]:
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are helpful asistant. You give only short answers"},
    {"role": "user", "content": "Tell a joke about AI."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="Why don't computers trust humans?\nBecause they always byte!", role='assistant', function_call=None, tool_calls=None)


##  pypdf - pdf loading

In [7]:
from pypdf import PdfReader

reader = PdfReader("data/cndx-etf-nsadaq100.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()

print(number_of_pages)
print(text)

4
CNDXiShares NASDAQ 100 UCITS ETF USD 
(Acc)
October Factsheet
Performance, Portfolio Breakdowns and Net Asset information as at: 31-Oct-
2023
All other data as at 06-Nov-2023
This document is marketing material.  For Investors in the UK. Investors should read the KIID/
PRIIPs document and prospectus prior to investing, and should refer to the prospectus for the 
funds full list of risks.
The Fund seeks to track the performance of an index composed of 100 of the largest non-financial 
companies listed on the NASDAQ Stock Market.
KEY BENEFITS
1 Targeted exposure to the largest non-financial stocks listed on The Nasdaq Stock Market
2 Direct investment into companies across major industry groups including computer 
hardware and software, telecommunications, retail/wholesale trade and biotechnology
3 International and large market capitalisation companies exposure
KEY RISKS: Investment risk is concentrated in specific sectors, countries, currencies or 
companies. This means the Fund is mo